In [ ]:
from django.contrib.gis.gdal import DataSource, GDALException

from django.contrib.gis.geos import LineString, MultiLineString
from assets.models import Road, SourceNationalRoad, SourceMunicipalRoad, SourceRrmpis, SourceR4D


def read_shape_file(shp_file, source_model, unique_keys):
    for feature in shp_file[0]:

        print('examining feature ', feature.fid, feature[unique_keys[0]])
        
        # try to get the source properties
        try:
            values = {key: feature[key] for key in unique_keys}
            properties = source_model.objects.get(**values)
        except source_model.MultipleObjectsReturned as ex:
            properties = source_model.objects.filter(**values).first()
            print('more than one match for ', ['{}={}'.format(key, val) for key, val in values.items()], 'using first found')
        except source_model.DoesNotExist as ex:
            print('empty row')
            continue

        # turn line strigns into multilinestrings
        try:
            if isinstance(feature.geom.geos, LineString):
                print('LineString - converting to multiline')
                multi_line_string = MultiLineString(feature.geom.geos)
            elif isinstance(feature.geom.geos, MultiLineString):
                print('MultiLineString - using as is')
                multi_line_string = feature.geom.geos
            else:
                print('!!!!! unhandled geometry type', feature.geom.geos)
        except GDALException as ex:
            print(dir(feature))
            print(feature[0])
            print('!!!!!!!', ex)
            break
            
        # create Road
        road = Road.objects.create(properties=properties, geom=multi_line_string)
        print('created', road)
        print('----------')


Road.objects.all().delete()
        
national_roads_shp = DataSource('../../ngis/National_Road.shp')
municipal_roads_shp = DataSource('../../ngis/Municipal_Road.shp')
rural_roads_r4d_shp = DataSource('../../ngis/Rural_Road_R4D_Timor_Leste.shp')
rural_roads_rrmpis_shp = DataSource('../../ngis/RRMPIS_2014.shp')
       
read_shape_file(national_roads_shp, SourceNationalRoad, ['name', 'subcode', 'code'])
read_shape_file(municipal_roads_shp, SourceMunicipalRoad, ['name', 'descriptio'])
read_shape_file(rural_roads_r4d_shp, SourceR4D, ['road_lin_1', 'id'])
#read_shape_file(rural_roads_rrmpis_shp, SourceRrmpis, ['rd_id', 'rdcode_cn', 'sheet_ref'])

print('we have', Road.objects.count(), 'roads')

In [21]:
Road.objects.all().delete()
SourceNationalRoadProperties

(116, {'assets.Road': 116})